In [9]:
import pandas as pd
import os
import glob

path_csv_train_TM="/data5/eloi/TencyMastering/PANNs_country_pop/train_split.csv"
path_csv_validation_TM="/data5/eloi/TencyMastering/PANNs_country_pop/val_split.csv"
path_csv_test_TM="/data5/eloi/TencyMastering/PANNs_country_pop/test_split.csv"

path_cav_train_tency="/home/eloi/projects/project_mfm_eloi/data/train_split_tency.csv"
path_csv_validation_tency="/home/eloi/projects/project_mfm_eloi/data/val_split_tency.csv"
path_csv_test_tency="/home/eloi/projects/project_mfm_eloi/data/test_split_tency.csv"


path_TencyDB="/data5/martinez/tency1_tency2_flac_48000k"

subdirs_train=["tency1_part0","tency1_part1","tency1_part2","tency1_part3","tency1_part4","tency1_part5","tency1_part6", "tency2_part1", "tency2_part2", "tency2_part3", "tency2_part4", "tency2_part5"]
subdirs_val=["tency1_part_valid", "tency2_part_valid"]
subdirs_test=[ "tency2_part_test"]


df_train = pd.DataFrame(columns=['id', 'path', 'subdir'])

ids_tency= []
for subdir in subdirs_train:
    path=os.path.join(path_TencyDB, subdir)
    songs= glob.glob(os.path.join(path, "*"))
    basename_songs = [os.path.basename(song) for song in songs]
    print(f"Found {len(basename_songs)} songs in {subdir}")
    #take last characters after the last found _ of the basename
    ids= [song.split('_')[-1] for song in basename_songs]
    ids_tency.extend(ids)

    for id, song in zip(ids, basename_songs):
        #add row to the dataframe with the id and path
        df_train = pd.concat([df_train, pd.DataFrame({'id': [id], 'path': [song], 'subdir': [subdir]})], ignore_index=True)

df_validation = pd.DataFrame(columns=['id', 'path', 'subdir'])

ids_tency_validation = []
for subdir in subdirs_val:
    path=os.path.join(path_TencyDB, subdir)
    songs= glob.glob(os.path.join(path, "*"))
    basename_songs = [os.path.basename(song) for song in songs]
    print(f"Found {len(basename_songs)} songs in {subdir}")
    #take last characters after the last found _ of the basename
    ids= [song.split('_')[-1] for song in basename_songs]
    ids_tency_validation.extend(ids)
    for id, song in zip(ids, basename_songs):
        #add row to the dataframe with the id and path
        #df_validation = df_validation.append({'id': id, 'path': song, 'subdir':subdir}, ignore_index=True)
        df_validation = pd.concat([df_validation, pd.DataFrame({'id': [id], 'path': [song], 'subdir': [subdir]})], ignore_index=True)


df_test = pd.DataFrame(columns=['id', 'path', 'subdir'])

ids_tency_test = []
for subdir in subdirs_test:
    path=os.path.join(path_TencyDB, subdir)
    songs= glob.glob(os.path.join(path, "*"))
    basename_songs = [os.path.basename(song) for song in songs]
    print(f"Found {len(basename_songs)} songs in {subdir}")
    #take last characters after the last found _ of the basename
    ids= [song.split('_')[-1] for song in basename_songs]
    ids_tency_test.extend(ids)

    for id, song in zip(ids, basename_songs):
        #add row to the dataframe with the id and path
        #df_test = df_test.append({'id': id, 'path': song, 'subdir':subdir}, ignore_index=True)
        df_test= pd.concat([df_test, pd.DataFrame({'id': [id], 'path': [song], 'subdir': [subdir]})], ignore_index=True)



df_TM_train= pd.read_csv(path_csv_train_TM)
df_TM_validation= pd.read_csv(path_csv_validation_TM)
df_TM_test= pd.read_csv(path_csv_test_TM)

#logic...

#I will concatenate df_TM_train with df_train
# However, there should be no duplicates in the ids between df_TM_train and df_train (in order to avoid duplicates in the final dataframe, conserve the one in df_TM_train)
# there should be no duplicates between df_TM_validation and df_train, or df_TM_test and df_train
# same for validation and test sets

#iterate over the rows of the dataframe
for index, row in df_TM_train.iterrows():
    id= row['id']
    id=os.path.basename(id)
    #replace id with the basename of the id
    df_TM_train.at[index, 'id'] = id

for index, row in df_TM_validation.iterrows():
    id= row['id']
    id=os.path.basename(id)
    #replace id with the basename of the id
    df_TM_validation.at[index, 'id'] = id

for index, row in df_TM_test.iterrows():
    id= row['id']
    id=os.path.basename(id)
    #replace id with the basename of the id
    df_TM_test.at[index, 'id'] = id


# First, let's check for duplicates between TM and Tency datasets
train_duplicates = set(df_TM_train['id']).intersection(set(df_train['id']))
val_duplicates = set(df_TM_validation['id']).intersection(set(df_validation['id']))
test_duplicates = set(df_TM_test['id']).intersection(set(df_test['id']))

#print the duplicates
print(f"Found {len(train_duplicates)} duplicates in train set between TM and Tency datasets")
print(f"Found {len(val_duplicates)} duplicates in validation set between TM and Tency datasets")
print(f"Found {len(test_duplicates)} duplicates in test set between TM and Tency datasets")

#now we can filter out the duplicates from the Tency datasets, keeping the ones from TM

df_train = df_train[~df_train['id'].isin(train_duplicates)]
df_validation = df_validation[~df_validation['id'].isin(val_duplicates)]
df_test = df_test[~df_test['id'].isin(test_duplicates)]

#now check for cross-set duplicates
train_val_duplicates= set(df_TM_train['id']).intersection(set(df_validation['id']))
train_test_duplicates= set(df_TM_train['id']).intersection(set(df_test['id']))
val_train_duplicates= set(df_TM_validation['id']).intersection(set(df_train['id']))
val_test_duplicates= set(df_TM_validation['id']).intersection(set(df_test['id']))
test_train_duplicates= set(df_TM_test['id']).intersection(set(df_train['id']))
test_val_duplicates= set(df_TM_test['id']).intersection(set(df_validation['id']))

print(f"Found {len(train_val_duplicates)} duplicates in train and validation sets between TM datasets")
print(f"Found {len(train_test_duplicates)} duplicates in train and test sets between TM datasets")
print(f"Found {len(val_train_duplicates)} duplicates in validation and train sets between TM datasets")
print(f"Found {len(val_test_duplicates)} duplicates in validation and test sets between TM datasets")
print(f"Found {len(test_train_duplicates)} duplicates in test and train sets between TM datasets")
print(f"Found {len(test_val_duplicates)} duplicates in test and validation sets between TM datasets")

df_validation = df_validation[~df_validation['id'].isin(train_val_duplicates)]
df_validation = df_validation[~df_validation['id'].isin(test_val_duplicates)]
df_test = df_test[~df_test['id'].isin(train_test_duplicates)]
df_test = df_test[~df_test['id'].isin(val_test_duplicates)]
df_train = df_train[~df_train['id'].isin(val_train_duplicates)]
df_train = df_train[~df_train['id'].isin(test_train_duplicates)]

#now concatenate the train dataframes
for index, row in df_TM_train.iterrows():
    id= row['id']
    song= row['path']
    subdir= "TencyMastering"
    df_train = pd.concat([df_train, pd.DataFrame({'id': [id], 'path': [song], 'subdir': [subdir]})], ignore_index=True)


#now save the dataframes to csv files
df_train.to_csv(path_cav_train_tency, index=False)
df_validation.to_csv(path_csv_validation_tency, index=False)
df_test.to_csv(path_csv_test_tency, index=False)

Found 5 songs in tency1_part0
Found 200 songs in tency1_part1
Found 200 songs in tency1_part2
Found 200 songs in tency1_part3
Found 199 songs in tency1_part4
Found 551 songs in tency1_part5
Found 100 songs in tency1_part6
Found 3926 songs in tency2_part1
Found 3920 songs in tency2_part2
Found 3927 songs in tency2_part3
Found 3901 songs in tency2_part4
Found 3926 songs in tency2_part5
Found 50 songs in tency1_part_valid
Found 200 songs in tency2_part_valid
Found 200 songs in tency2_part_test
Found 2 duplicates in train set between TM and Tency datasets
Found 0 duplicates in validation set between TM and Tency datasets
Found 0 duplicates in test set between TM and Tency datasets
Found 1 duplicates in train and validation sets between TM datasets
Found 0 duplicates in train and test sets between TM datasets
Found 2 duplicates in validation and train sets between TM datasets
Found 0 duplicates in validation and test sets between TM datasets
Found 0 duplicates in test and train sets between

In [8]:
ids_tency

['49375',
 '48052',
 '51228',
 '51691',
 '51837',
 '44890',
 '35886',
 '24751',
 '5942',
 '30288',
 '47559',
 '40948',
 '19580',
 '43655',
 '29948',
 '33975',
 '6080',
 '43953',
 '45730',
 '9889',
 '38004',
 '51452',
 '37503',
 '32435',
 '38363',
 '22647',
 '32251',
 '9821',
 '48433',
 '34936',
 '31722',
 '8038',
 '21964',
 '15505',
 '15292',
 '13899',
 '45756',
 '20951',
 '14673',
 '49331',
 '50117',
 '32531',
 '47765',
 '49166',
 '28945',
 '48475',
 '29828',
 '24706',
 '28801',
 '16682',
 '39578',
 '35663',
 '35374',
 '48352',
 '29097',
 '17145',
 '41641',
 '24233',
 '34234',
 '25649',
 '10168',
 '31044',
 '39306',
 '5407',
 '12093',
 '41443',
 '36438',
 '27487',
 '46699',
 '26794',
 '22364',
 '13025',
 '35835',
 '44217',
 '41972',
 '38570',
 '5130',
 '10593',
 '25683',
 '40287',
 '44758',
 '11730',
 '48175',
 '47856',
 '45289',
 '21899',
 '35250',
 '13787',
 '45565',
 '49543',
 '46158',
 '49694',
 '28111',
 '31157',
 '47201',
 '16501',
 '11427',
 '48646',
 '39454',
 '41750',
 '49589